In [2]:
!pip install mediapipe opencv-python

In [3]:
import mediapipe as mp
import cv2

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [5]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Raw Webcam Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
cap.release()
cv2.destroyAllWindows()

In [7]:
pip install mediapipe --upgrade


Note: you may need to restart the kernel to use updated packages.


In [8]:
import mediapipe as mp

print(dir(mp.solutions.holistic))


['FACEMESH_CONTOURS', 'FACEMESH_TESSELATION', 'HAND_CONNECTIONS', 'HandLandmark', 'Holistic', 'NamedTuple', 'POSE_CONNECTIONS', 'PoseLandmark', 'SolutionBase', '_BINARYPB_FILE_PATH', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_download_oss_pose_landmark_model', 'constant_side_packet_calculator_pb2', 'detections_to_rects_calculator_pb2', 'download_utils', 'gate_calculator_pb2', 'image_to_tensor_calculator_pb2', 'inference_calculator_pb2', 'landmark_projection_calculator_pb2', 'local_file_contents_calculator_pb2', 'non_max_suppression_calculator_pb2', 'np', 'rect_transformation_calculator_pb2', 'roi_tracking_calculator_pb2', 'split_vector_calculator_pb2', 'ssd_anchors_calculator_pb2', 'switch_container_pb2', 'tensors_to_classification_calculator_pb2', 'tensors_to_floats_calculator_pb2', 'tensors_to_landmarks_calculator_pb2']


In [9]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)

# Initialize Holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break
        
        # Convert the image to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the image and get the results
        results = holistic.process(image_rgb)
        
        # Convert the image back to BGR for display
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        
        # Detecting SOS based on both hands
        if results.right_hand_landmarks and results.left_hand_landmarks:
            right_landmarks = results.right_hand_landmarks.landmark
            left_landmarks = results.left_hand_landmarks.landmark
            
            # Check if both index fingers are raised above other landmarks
            if (right_landmarks[8].y < right_landmarks[5].y and right_landmarks[8].y < right_landmarks[4].y and
                left_landmarks[8].y < left_landmarks[5].y and left_landmarks[8].y < left_landmarks[4].y):
                gesture_label = "SOS Detected"
                color = (0, 255, 0)  # Green
            else:
                gesture_label = "Not SOS"
                color = (0, 0, 255)  # Red
            
            # Draw gesture label
            cv2.putText(image_bgr, gesture_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, color, 2, cv2.LINE_AA)
        
        # Draw landmarks if present
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(image_bgr, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(image_bgr, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        # Show the frame with landmarks
        cv2.imshow('Raw Webcam Feed', image_bgr)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


C:\Users\abhin\OneDrive\Documents\Anaconda\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [10]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [13]:
import cv2
import base64
import requests
from datetime import datetime
import time

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Flask server URL
FLASK_SERVER_URL = 'http://127.0.0.1:5000/alert'

# Geocoding API URL
GEOCODE_API_URL = 'https://nominatim.openstreetmap.org/reverse'

# Function to get address from latitude and longitude
def get_address_from_coordinates(lat, lon):
    try:
        response = requests.get(GEOCODE_API_URL, params={
            'lat': lat,
            'lon': lon,
            'format': 'json',
            'addressdetails': 1
        })
        data = response.json()
        if 'address' in data:
            address = data['address']
            formatted_address = ', '.join([
                address.get('road', ''),
                address.get('suburb', ''),
                address.get('city', ''),
                address.get('state', ''),
                address.get('postcode', ''),
                address.get('country', '')
            ]).strip(', ')
            return formatted_address or 'Address not found'
        else:
            return 'Address not found'
    except Exception as e:
        print(f"Error fetching address: {e}")
        return 'Address lookup failed'

# Function to encode image to base64
def encode_image_to_base64(image):
    _, buffer = cv2.imencode('.jpg', image)
    return base64.b64encode(buffer).decode('utf-8')

# Coordinates
latitude = 12.963829
longitude = 77.505777

# Get address for the provided coordinates
address = get_address_from_coordinates(latitude, longitude)
print(f"Address: {address}")

# Open webcam
cap = cv2.VideoCapture(0)

# Initialize Holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    sos_alert_sent = False
    display_sos = False
    sos_display_start_time = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        # Convert the image to RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the image and get the results
        results = holistic.process(image_rgb)
        
        # Convert the image back to BGR for display
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Detecting SOS based on both hands
        if results.right_hand_landmarks and results.left_hand_landmarks:
            right_landmarks = results.right_hand_landmarks.landmark
            left_landmarks = results.left_hand_landmarks.landmark
            
            if (right_landmarks[8].y < right_landmarks[5].y and right_landmarks[8].y < right_landmarks[4].y and
                left_landmarks[8].y < left_landmarks[5].y and left_landmarks[8].y < left_landmarks[4].y):
                
                gesture_label = "SOS Detected"
                color = (0, 255, 0)  # Green
                
                if not sos_alert_sent:
                    # Encode the captured image
                    encoded_image = encode_image_to_base64(frame)

                    # Prepare data to send to Flask with the provided location
                    alert_data = {
                        "message": "SOS",
                        "location": {
                            "latitude": latitude,
                            "longitude": longitude,
                            "address": address
                        },
                        "timestamp": datetime.now().isoformat(),
                        "image": encoded_image
                    }
                    
                    # Send SOS alert to Flask server
                    try:
                        response = requests.post(FLASK_SERVER_URL, json=alert_data)
                        print("Alert sent:", response.json())
                        sos_alert_sent = True
                        display_sos = True
                        sos_display_start_time = time.time()
                    except Exception as e:
                        print(f"Failed to send alert: {e}")
        
        if display_sos:
            cv2.putText(image_bgr, "SOS Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (0, 255, 0), 2, cv2.LINE_AA)
            if time.time() - sos_display_start_time > 2:
                break

        else:
            gesture_label = "Not SOS"
            color = (0, 0, 255)  # Red
            cv2.putText(image_bgr, gesture_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, color, 2, cv2.LINE_AA)
        
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(image_bgr, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(image_bgr, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        cv2.imshow('Raw Webcam Feed', image_bgr)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Address: Malathhalli Main Road, Mallathahalli, Bengaluru, Karnataka, 560072, India
Alert sent: {'status': 'Alert received'}
